Now Reading The Train Data

In [ ]:
! kill -9 -1

In [2]:
import sys
!{sys.executable} -m pip install torch

  Using cached https://files.pythonhosted.org/packages/58/26/6b86448bda61c1ce463ad2c962641a933113f4496de16085df41217cdccc/torch-1.6.0-cp37-none-macosx_10_9_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/45/0b/38b06fd9b92dc2b68d58b75f900e97884c45bedd2ff83203d933cf5851c9/future-0.18.2.tar.gz
  Running setup.py install for future ... done
You are using pip version 19.0.3, however version 20.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [24]:
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
from torch.autograd import Variable

In [4]:
file_name = '/Users/yannangao/PycharmProjects/sml/data/train.txt'

train_data = []
with open(file_name, 'r') as file:
    for line in tqdm(file):
        train_data.append(line)
        
print(len(train_data))

20000it [00:00, 29430.34it/s]

20000


In [5]:
print(train_data[0])

540762	1912140	1537559	3091331	2757277	3237295	1070876	4008078	1824878	1005927	2703564	2519640	3370407	3883660	2173241	1726071	2451654	3906434	2635670	3294915	2763417	1116703	261881	2404108	3521384	2299939	2989624	4361289	3314815	3367669	641925	2862019	1274046	4316209	2463912	3043	952976	3532986	415955	4366854	267263	1199298	1840280	581535	4241326	4267183	2881471	2737722	4286284	333573	1594907	3056533	2163471	3712732	2672578	1209171	1003239	4818002	3201437	1892251	2800259	141083	2558493	1655093	4313335	738638	3702208	1124524	1436216	3206499	1462439	2345899	4131275	1169783	2152173	4174165	141067	3118623	1521082	3912913	299865	193736	3562621	94742	2464827	1719625	557514	1313296	2329535	3662772	3980998	695775	381025	3349332	3704614	1392121	3767319	1157186	4230834	1411138	1312847	1818911	3820661	1927780	400117	1853195	3994838	1327933	3386592	3766535	343833	4582129	892512	3316576	3011333	2673351	746711	2770509	3670190	4208653	1970076	2183328	2492020	289918	1436562	2335027	3133671	977117	851

In [6]:
# store the root user and his follows
from collections import defaultdict

pair_list = []

for item in train_data:
    temp_data = item.split()
    if len(temp_data) > 1:
        follows_list = []
        for i in range(len(temp_data)-1):
            follows_list.append(temp_data[i+1])
        pair_list.append((temp_data[0], follows_list))
    else:
        pair_list.append((temp_data[0], []))

print(len(pair_list))

20000


In [7]:
# remove the node with no follows and change the id to '<unk>' if follows no greater than 5

new_node_pair = []
unk_follow_list = []

for pair in tqdm(pair_list):
    if len(pair[1]) > 0:
      if len(pair[1]) <= 5:
        new_node_pair.append(('unk', pair[1]))
        unk_follow_list.append(pair[1])
      elif len(pair[1]) <= 100:
        new_node_pair.append((pair[0], pair[1]))
      else:
        new_node_pair.append((pair[0], pair[1][:100]))

print('\nThe Node pairs length is {}'.format(len(new_node_pair)))

100%|██████████| 20000/20000 [00:00<00:00, 165857.83it/s]


The Node pairs length is 19570


In [8]:
unk_follows = []
for item in unk_follow_list:
    for follow in item:
        unk_follows.append(follow)

print('The number of unk follows is {}'.format(len(unk_follows)))

The number of unk follows is 1795


In [9]:
# Generate the negative samples

from collections import defaultdict

def get_neg_data(original_data, length):
    neg_dict = defaultdict(list)
    count = 0

    for item in original_data:
        temp_list = []

        while len(temp_list) < length:
            random_num = random.randint(0, 5000000)
            if item[0] == '<unk>':
               if str(random_num) not in unk_follows:
                  temp_list.append(str(random_num))
            elif str(random_num) != item[0] and str(random_num) not in item[1]:
                temp_list.append(str(random_num))
        neg_dict[item[0]] = temp_list
        count += 1
        if count % 100 == 0:
            print('Now process {} files!'.format(count))
    
    return neg_dict

neg_dict = get_neg_data(new_node_pair, 20)
print(len(neg_dict))  



Now process 100 files!
Now process 200 files!
Now process 300 files!
Now process 400 files!
Now process 500 files!
Now process 600 files!
Now process 700 files!
Now process 800 files!
Now process 900 files!
Now process 1000 files!
Now process 1100 files!
Now process 1200 files!
Now process 1300 files!
Now process 1400 files!
Now process 1500 files!
Now process 1600 files!
Now process 1700 files!
Now process 1800 files!
Now process 1900 files!
Now process 2000 files!
Now process 2100 files!
Now process 2200 files!
Now process 2300 files!
Now process 2400 files!
Now process 2500 files!
Now process 2600 files!
Now process 2700 files!
Now process 2800 files!
Now process 2900 files!
Now process 3000 files!
Now process 3100 files!
Now process 3200 files!
Now process 3300 files!
Now process 3400 files!
Now process 3500 files!
Now process 3600 files!
Now process 3700 files!
Now process 3800 files!
Now process 3900 files!
Now process 4000 files!
Now process 4100 files!
Now process 4200 files!
N

In [10]:
def get_id_pair(data_list):
    pairs_list = []
    for item in data_list:
        for follows in item[1]:
            pairs_list.append((item[0], follows))

    return pairs_list

pos_pair = get_id_pair(new_node_pair) # All the node Pair

print(len(pos_pair))

1540302


In [11]:
# Now Create positive train samples and dev samples
from sklearn.model_selection import train_test_split

train_pos_pair, dev_pos_pair = train_test_split(pos_pair, test_size=0.2, random_state=42)


print('The length of train pos pair is {}'.format(len(train_pos_pair)))
print('The length of dev pos pair is {}'.format(len(dev_pos_pair)))


The length of train pos pair is 1232241
The length of dev pos pair is 308061


In [12]:
# Build up a node set
node_set = set()

for pair in train_pos_pair:
    node_set.add(pair[0])
    node_set.add(pair[1])
      
print(len(node_set))


544026


In [13]:
# Build Up node2idx dict in the format as node2idx['123456'] = 123(index)
node2idx = {}
node2idx['<unk>'] = 0

for i, item in enumerate(node_set):
    node2idx[item] = i + 1

print(len(node2idx))
print(node2idx['<unk>'])

544027
0


In [14]:
# Each pos sample give a negtive sample

def build_neg_list(pos_pairs):
  neg_list = []
  for pair in tqdm(pos_pairs):
      temp_list = []
      neg_samples = random.choice(neg_dict[pair[0]])
      neg_list.append(neg_samples)

  return neg_list

train_neg_list = build_neg_list(train_pos_pair)

print('The length of train neg list is {}'.format(len(train_neg_list)))


100%|██████████| 1232241/1232241 [00:02<00:00, 492120.28it/s]

The length of train neg list is 1232241


In [15]:
# Now create the dev neg samples and dev labels so that to assess the model

def create_neg_pair(dev_data):
    neg_list = []
    for pair in dev_data:
        neg_sample = random.choice(neg_dict[pair[0]])
        neg_list.append((pair[0], neg_sample))

    return neg_list


dev_pos_pair = random.sample(dev_pos_pair, 1000)
dev_neg_pair = create_neg_pair(dev_pos_pair)
print(len(dev_neg_pair))

dev_pair = dev_pos_pair + dev_neg_pair
print('The length of dev pairs is {}'.format(len(dev_pair)))
dev_label = [1] * len(dev_pos_pair) + [0] * len(dev_neg_pair)
print('The length of dev label is {}'.format(len(dev_label))) 

1000
The length of dev pairs is 2000
The length of dev label is 2000


In [16]:
# combine the dev pair and dev label
dev_data = []
for i in range(len(dev_pair)):  
    dev_data.append((dev_pair[i], dev_label[i]))

print('The length of dev_data is {}'.format(len(dev_data)))
print('The first component if dev_data is {}'.format(dev_data[0]))

# Now we will shuffle the dev_data and pick 2000 samples
random.shuffle(dev_data)
print('Now we got a {} length dev pairs and labels'.format(len(dev_data)))

The length of dev_data is 2000
The first component if dev_data is (('3739473', '2635123'), 1)
Now we got a 2000 length dev pairs and labels


In [17]:
# transfer the id pair into index pair
def build_up_vec(data):
    index_pair = []
    for pair in data:
       pair_list = []
       if pair[0] in node2idx.keys():
          index1 = node2idx[pair[0]]
       else:
          index1 = node2idx['<unk>']
       pair_list.append(index1)
       if pair[1] in node2idx.keys():
          index2 = node2idx[pair[1]]
       else:
         index2 = node2idx['<unk>']
       pair_list.append(index2)
       index_pair.append(pair_list)
    
    return index_pair

train_idx_data = build_up_vec(train_pos_pair)

print('The length of train_idx_data is {}'.format(len(train_idx_data)))

The length of train_idx_data is 1232241


In [ ]:
# transfer the id pair into index pair
def build_up_vec(data):
    index_pair = []
    for pair in data:
       pair_list = []
       if pair[0] in node2idx.keys():
          index1 = node2idx[pair[0]]
       else:
          index1 = node2idx['<unk>']
       pair_list.append(index1)
       if pair[1] in node2idx.keys():
          index2 = node2idx[pair[1]]
       else:
         index2 = node2idx['<unk>']
       pair_list.append(index2)
       index_pair.append(pair_list)
    
    return index_pair

train_idx_data = build_up_vec(train_pos_pair)

print('The length of train_idx_data is {}'.format(len(train_idx_data)))
      

torch.Size([1369257, 2])
torch.Size([342315, 2])


In [18]:
# Convert neg ids in to index
train_neg_idx = []
for neg in tqdm(train_neg_list):
    if neg in node2idx.keys():
        train_neg_idx.append(node2idx[neg])
    else:
        train_neg_idx.append(node2idx['<unk>'])


print('The length of train negative sample list is {}'.format(len(train_neg_idx)))
print('The first component of train_neg_list is {}'.format(train_neg_idx[0]))


100%|██████████| 1232241/1232241 [00:01<00:00, 1121929.41it/s]

The length of train negative sample list is 1232241
The first component of train_neg_list is 0


In [19]:
vocabulary_size = len(node2idx)

def look_up_table(word_idx):
    x = torch.zeros(vocabulary_size).float()
    x[word_idx] = 1.0
    return x

Now Build Up Our Model

In [20]:
N = 5
num_epochs = 20
learning_rate = 0.001
embedding_dims = 50

torch.cuda.empty_cache()

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("There are {} GPUs available.".format(torch.cuda.device_count()))
    print("We will use GPU {}".format(torch.cuda.get_device_name(0)))
else:
    print("There is no GPU available, using the CPU instead!")
    device = torch.device("cpu")

There is no GPU available, using the CPU instead!


In [21]:
from numpy import dot
from numpy.linalg import norm

def cosine_sim(a, b):
    return abs(dot(a, b)/(norm(a)*norm(b)))


In [22]:
# Function to calculate the accuracy of our predictions vs labels
def accuracy(preds, y):
    """
    returns accuracy per batch
    """
    correct = (class_preds == y).astype(float) # convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [27]:
# The two weight matrices:
W1 = torch.randn(embedding_dims, vocabulary_size, requires_grad=True)
W2 = torch.randn(embedding_dims, vocabulary_size, requires_grad=True)


W1 = Variable(W1.to(device), requires_grad=True)
W2 = Variable(W2.to(device), requires_grad=True)

for epoch in range(num_epochs):
    epoch_loss = 0

    for i in tqdm(range(len(train_idx_data))):

        data = train_idx_data[i][0]
        target = train_idx_data[i][1]
        

        x_var = Variable(look_up_table(data)).float()
        y_pos_var = Variable(look_up_table(target)).float()

        # pick a negative sample
        neg_sample = train_neg_idx[i]
        y_neg_var = Variable(look_up_table(neg_sample)).float()
        # use the look up table

        x_emb = torch.matmul(W1, x_var)
        y_pos_emb = torch.matmul(W2, y_pos_var)
        y_neg_emb = torch.matmul(W2, y_neg_var)


        # get positive sample score
        pos_loss = nn.functional.logsigmoid(torch.matmul(x_emb, y_pos_emb))
        
        # get negative sample score
        neg_loss = nn.functional.logsigmoid(-1 * torch.matmul(x_emb, y_neg_emb))
        
        # TO DO: compute loss
     
        loss = - (pos_loss + neg_loss.sum())

    
        epoch_loss += loss.item()
        
        # propagate the error
        loss.backward()
        
        # gradient descent
        W1.data -= learning_rate * W1.grad.data
        W2.data -= learning_rate * W2.grad.data

        # zero out gradient accumulation
        W1.grad.data.zero_()
        W2.grad.data.zero_()


    print(f'Loss at epoch {epoch}: {epoch_loss/len(train_idx_data)}')

    preds = []
    labels = []
    for dev_data in tqdm(dev_pair):
        dev_id_pair = dev_data[0]
        dev_label = dev_data[1]
        labels.append(dev_label)

        dev_idx = [node2idx[node_id] for node_id in dev_id_pair]

        with torch.no_grad():
            dev_x_var = look_up_table(dev_idx[0]).float()
            dev_y_pos = look_up_table(dev_idx[1]).float()

            dev_x_emb = torch.matmul(W1, x_var)
            dev_y_pos_emb = torch.matmul(W2, y_pos_var)

            x_vec = dev_x_emb.numpy()
            y_vec = dev_y_pos_emb.numpy()

            cosine_score = cosine_sim(x_vec, y_vec)

            if cosine_score > 0.5:
                preds.append(1)
            else:
                preds.append(0)
    
    dev_accuracy = accuracy(np.array(preds), np.array(labels))

    print(f'The accyracy on the dev data is {dev_accuracy:.4f}')

print('Training Process Complete!')  

        
    

  0%|          | 5046/1232241 [5:39:44<1377:05:16,  4.04s/it]    


KeyboardInterrupt: 

Now we perform analysis on Confusion Matrix

In [ ]:
def list2vec(data_list):
    temp_list = []
    for item in data_list:
      temp_list.extend(list(item))

    return np.array(temp_list)

val_pred_arr = list2vec(best_val_preds)
val_labels_arr - list2vec(best_val_labels)

print(val_pred_arr.shape)
print(val_labels_arr.shape)

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(val_labels_arr, val_pred_arr)

def plot_confusion_matrix(cm, classes, title='Confusion matrix', cmap=plt.cm.Blues):
    print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

plt.figure(figsize=(5,5))
class_list = ['positive', 'negative']
plot_confusion_matrix(cm, class_list)

In [ ]:
# Read the Test set

test_file = 'test-public.txt'

id_list = []
root_list = []
follows_list = []

with open(test_file, "r") as file:
  next(file)
  for line in file:
      line = line.strip().split()
      id_list.append(line[0])
      root_list.append(line[1])
      follows_list.append(line[2])

print(len(id_list))
print(len(root_list))
print(len(follows_list))

In [ ]:
def id2idx(data):
   idx = []
   for id in data:
     if id in node2idx.keys():
          index = node2idx[id]
     else:
          index = node2idx['<unk>']

     idx.append(index)
  
   return idx

root_arr = id2idx(root_list)
follows_arr = id2idx(follows_list)

print(root_arr.shape)
print(follows_arr.shape)

(2000,)
(2000,)


In [ ]:
test_pair = []

for i in range(len(root_arr)):
    test_pair.append([root_arr[i], follows_arr[i]])

print(len(test_pair))

test_tensor = torch.LongTensor(test_pair)

print(test_tensor.shape)


2000
torch.Size([2000, 2])


In [ ]:
test_batch_size = 100

test_data = TensorDataset(test_tensor)
test_loader = DataLoader(test_data, batch_size=test_batch_size, shuffle=False)
print(len(test_loader))

20


In [ ]:
from scipy.special import softmax

def get_result(test_loader, model):
    result = []
    model.eval()
    with torch.no_grad():
        for batch in test_loader:

            test_input = batch[0].cuda()
  
            outputs = model(test_input)
            preds = outputs.detach().cpu().numpy()

            pred_result = np.argmax(softmax(preds), axis=1)
            for score in pred_result:
              result.append(score)
    return np.array(result)

pred = get_result(test_loader, model)
print(len(pred))

2000


In [ ]:
score_dict = {'Id':id_list, 'Predicted':pred}
score_df = pd.DataFrame(score_dict)
score_df.to_csv('submission.csv', index=False)